In [4]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_lattice as tfl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.random.seed(1)

In [5]:
# DATA
df = pd.read_csv('C:/Users/tln229/Downloads/Python/Building/data/HVAC_B90_102_exp_10m_20210424.csv')

In [6]:
f = open('C:/Users/tln229/Downloads/Python/Building/1. Zone subsystem/stat2.txt', 'w')

for n in ([16, 32, 64, 128]):
    for y_max in ([2, 5, 10, 20]):
        for grid_size in ([5, 10, 20, 40]):
            for lr in ([0.001, 0.005, 0.1, 0.2]):

                n_train = n

                # trainning set
                Tz_k   = np.array(df['room_temp']).reshape(-1,1)[0: n_train]
                Tsa_k1 = np.array(df['supply_discharge_temp']).reshape(-1,1)[1: n_train+1]
                msa_k1 = np.array(df['airflow_current']).reshape(-1,1)[1: n_train+1]
                Tz_k1  = np.array(df['room_temp']).reshape(-1,1)[1: n_train+1]
                z      = ((msa_k1-400)/600)*(Tsa_k1-Tz_k)

                X = np.concatenate((Tz_k, (Tsa_k1-65)/20, z), axis=1)
                Y = np.copy(Tz_k1)

                # test set
                Tz_k_t   = np.array(df['room_temp']).reshape(-1,1)[n_train: 1600]
                Tsa_k1_t = np.array(df['supply_discharge_temp']).reshape(-1,1)[n_train+1: 1600+1]
                msa_k1_t = np.array(df['airflow_current']).reshape(-1,1)[n_train+1: 1600+1]
                Tz_k1_t  = np.array(df['room_temp']).reshape(-1,1)[n_train+1: 1600+1]
                z_t      = ((msa_k1_t-400)/600)*(Tsa_k1_t-Tz_k_t)

                X_t = np.concatenate((Tz_k_t, (Tsa_k1_t-65)/20, z_t), axis=1)
                Y_t = np.copy(Tz_k1_t)

                # combined calibrator
                combined_calibrators = tfl.layers.ParallelCombination()

                calibrator_x1 = tfl.layers.PWLCalibration(input_keypoints=np.linspace(65, 85, grid_size), output_min=0, output_max=y_max)
                calibrator_x2 = tfl.layers.PWLCalibration(input_keypoints=np.linspace(-1, 2,  grid_size), output_min=0, output_max=y_max, monotonicity='increasing')
                calibrator_x3 = tfl.layers.PWLCalibration(input_keypoints=np.linspace(-1, 20, grid_size), output_min=0, output_max=y_max)

                combined_calibrators.append(calibrator_x1)
                combined_calibrators.append(calibrator_x2)
                combined_calibrators.append(calibrator_x3)

                # Lattice
                lattice = tfl.layers.Lattice(
                    lattice_sizes = [y_max+1, y_max+1, y_max+1],
                    monotonicities = ['increasing', 'increasing', 'increasing'],
                    output_min = 65,
                    output_max = 85)
                
                # model
                inps  = tf.keras.Input(shape=(3,))
                calib = combined_calibrators(inps)
                outs  = lattice(calib)

                model = tf.keras.Model(inputs=inps, outputs=outs)
                model.compile(
                    loss = tf.keras.losses.mean_squared_error,
                    optimizer = tf.keras.optimizers.Adam(learning_rate=lr),
                    metrics = [tfa.metrics.RSquare()])

                callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)

                hist = model.fit(
                    x = X,
                    y = Y,
                    batch_size = 256,
                    epochs = 200000,
                    shuffle = False,
                    validation_data=(X_t, Y_t),
                    callbacks=[callback],
                    verbose = 0)
                
                a = np.argmax(hist.history['val_r_square'])
                print('n: %3d  -  y_max: %2d  -  grid_size: %3d  -  lr:%6.4f  -  r2: %7.4f' % (n, y_max, grid_size, lr, hist.history['val_r_square'][a]))

                with open('C:/Users/tln229/Downloads/Python/Building/1. Zone subsystem/stat2.txt', 'a') as f:
                    f.write('n: %3d  -  y_max: %2d  -  grid_size: %3d  -  lr:%6.4f  -  r2: %7.4f \n' % (n, y_max, grid_size, lr, hist.history['val_r_square'][a]))

n:  16  -  y_max:  2  -  grid_size:   5  -  lr:0.0010  -  r2: -0.9320
n:  16  -  y_max:  2  -  grid_size:   5  -  lr:0.0050  -  r2: -0.9674
n:  16  -  y_max:  2  -  grid_size:   5  -  lr:0.1000  -  r2:  0.6580
n:  16  -  y_max:  2  -  grid_size:   5  -  lr:0.2000  -  r2:  0.7644
n:  16  -  y_max:  2  -  grid_size:  10  -  lr:0.0010  -  r2: -0.9349
n:  16  -  y_max:  2  -  grid_size:  10  -  lr:0.0050  -  r2: -0.9838
n:  16  -  y_max:  2  -  grid_size:  10  -  lr:0.1000  -  r2: -0.2572
n:  16  -  y_max:  2  -  grid_size:  10  -  lr:0.2000  -  r2:  0.6532
n:  16  -  y_max:  2  -  grid_size:  20  -  lr:0.0010  -  r2: -0.9392
n:  16  -  y_max:  2  -  grid_size:  20  -  lr:0.0050  -  r2: -1.0101
n:  16  -  y_max:  2  -  grid_size:  20  -  lr:0.1000  -  r2: -0.1191
n:  16  -  y_max:  2  -  grid_size:  20  -  lr:0.2000  -  r2:  0.0402
n:  16  -  y_max:  2  -  grid_size:  40  -  lr:0.0010  -  r2: -0.9506
n:  16  -  y_max:  2  -  grid_size:  40  -  lr:0.0050  -  r2: -1.0831
n:  16  -  y_max:  2